<a href="https://colab.research.google.com/github/jayaraam23/twitter-sentiment-analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install kaggle


In [8]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 78% 63.0M/80.9M [00:00<00:00, 146MB/s]
100% 80.9M/80.9M [00:00<00:00, 144MB/s]


In [9]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset , 'r') as zip:
  zip.extractall()
  print('THe Dataset is extracted')

THe Dataset is extracted


In [34]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [14]:
twitter_data  = pd.read_csv('/content/training_data.csv' , encoding = 'ISO-8859-1')

In [15]:
twitter_data.shape

(1599999, 6)

In [16]:
column_names = ['target' , 'id' ,'date', 'flag' , 'user' , 'text']
twitter_data = pd.read_csv('/content/training_data.csv', names=column_names , encoding='ISO-8859-1')


In [17]:
twitter_data.shape

(1600000, 6)

In [18]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [19]:
twitter_data.replace({'target':{4:1}} , inplace =True)

In [20]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 ---  negtive
1 ---  postive

In [21]:
port_stem = PorterStemmer()

In [22]:
def stemming(content ):
  stemmed_content = re.sub ('[^a-zA-A]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [23]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [25]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer ou ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat acebook text might cri result choo...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,enichan dive mani time ball anag save rest go ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [26]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer ou ...
1          upset updat acebook text might cri result choo...
2          enichan dive mani time ball anag save rest go ...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                   ust woke ave school best feel ever
1599996    com eri cool hear old alt interview http blip ...
1599997                               readi akeov ask detail
1599998       appi th irthday boo alll time upac amaru hakur
1599999              happi charitytuesday park hariti peak p
Name: stemmed_content, Length: 1600000, dtype: object


In [24]:
X = twitter_data [ 'stemmed_content' ].values
Y = twitter_data['target'].values

In [27]:
print(X)

['switchfoot http twitpic com zl awww bummer ou shoulda got avid arr hird ay'
 'upset updat acebook text might cri result chool today also lah'
 'enichan dive mani time ball anag save rest go bound' ...
 'readi akeov ask detail' 'appi th irthday boo alll time upac amaru hakur'
 'happi charitytuesday park hariti peak p']


In [28]:
X_train , X_test , Y_train, Y_test = train_test_split(X ,Y , test_size = 0.2, stratify=Y , random_state = 2)

In [29]:
print(X.shape , X_test.shape , X_train.shape)

(1600000,) (320000,) (1280000,)


In [30]:
print(X_train)

['watch saw iv drink lil wine' 'ater agazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager onday afternoon'
 'ope everyon mother great day wait hear guy store tomorrow'
 'love wake olger oo bad voic deeper']


In [31]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [32]:
print(X_train)

  (0, 390643)	0.44095783406829225
  (0, 212912)	0.42097598581070167
  (0, 98626)	0.3764542685581735
  (0, 176577)	0.5276155164827365
  (0, 317705)	0.36080806071416366
  (0, 385471)	0.2793868316853507
  (1, 7404)	0.7616797059973436
  (1, 34328)	0.6479537217053393
  (2, 252264)	0.17045229485829705
  (2, 125916)	0.20099683770133864
  (2, 139390)	0.18906288920472053
  (2, 167553)	0.16473058638547
  (2, 358385)	0.15228244554707487
  (2, 236429)	0.23990008245570715
  (2, 390928)	0.3319321005528492
  (2, 75355)	0.3177658015978687
  (2, 382982)	0.3309903805499405
  (2, 356271)	0.3203450732902036
  (2, 123736)	0.287563592665274
  (2, 356975)	0.18508165034903243
  (2, 119215)	0.18903540185930914
  (2, 98626)	0.4592508128933
  (3, 159688)	0.3711930299322745
  (3, 360257)	0.27223958996866227
  (3, 343100)	0.3969994257133483
  :	:
  (1279996, 286027)	0.22931666841005363
  (1279996, 215023)	0.24179956117434154
  (1279996, 254400)	0.1925925322189422
  (1279996, 238487)	0.3623793065226056
  (1279996, 

In [35]:
model = LogisticRegression(max_iter=1000)

In [36]:
model.fit(X_train , Y_train)

LogisticRegression(max_iter=1000)

In [37]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train , X_train_prediction)

In [38]:
print(training_data_accuracy)

0.8013078125


In [40]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test , X_test_prediction)

In [41]:
print(test_data_accuracy)

0.76904375


In [42]:
import pickle

In [43]:
filename = 'trianed_model.sav'
pickle.dump(model,open(filename ,'wb'))

In [46]:
loaded_model = pickle.load(open('/content/trained_model.sav' , 'rb'))

In [49]:
X_new = X_test[200]
print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

1
[1]
Positive Tweet


In [50]:
X_new = X_test[3]
print(Y_test[3])

prediction = loaded_model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[0]
Negative Tweet


# New Section